In [ ]:
from keras.models import load_model

model = load_model('./Model/ocnn-x-day.h5')

In [ ]:
K = len(os.listdir(validation_dir+'/'+os.listdir(validation_dir)[0])) + len(os.listdir(validation_dir+'/'+os.listdir(validation_dir)[1]))
print(K)

import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import axes
from matplotlib.font_manager import FontProperties
from sklearn.metrics import roc_curve, auc
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
from PIL import Image

batch_size=40
validation_labels = np.zeros(shape=(K,2))
validation_generator2 = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
i = 0
for ____, labels_batch in validation_generator2:
    validation_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= K:
        break

preds = model.predict_generator(validation_generator2, steps=int(np.ceil(K/batch_size)))

import tensorflow.compat.v1 as tf

tf.compat.v1.disable_eager_execution()
x = tf.strided_slice(preds,[0,1],[K+1,2])
y = tf.strided_slice(validation_labels,[0,1],[K+1,2])
x1 = tf.strided_slice(preds,[0,0],[K+1,1])
y1 = tf.strided_slice(validation_labels,[0,0],[K+1,1])

with tf.Session() as sess:
    preds2 = sess.run(x)
    validation_labels2 = sess.run(y)
    preds3 = sess.run(x1)
    validation_labels3 = sess.run(y1)
    
preds4 = np.reshape(preds3, (len(preds3)))
preds5 = np.reshape(preds2, (len(preds2)))
headers = ['Names','Labels','Predictions0','Predictions1']
image_names = [name for name in os.listdir('./XXX/TEST/'+'1') for item in IMAGES_FORMAT if os.path.splitext(name)[1] == item]
with open(SavePath+'Predictions/xxx_TEST/Predictions_test'+ModelName+'.csv','w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(preds2)):
        f_csv.writerow([image_names[i],train_labels[i],preds4[i],preds5[i]])